# Impact Quantization

In [1]:
# imports
import os
import numpy as np
import pickle

In [2]:
# load impact points
load_dir = os.path.join('data', 'impact_points')
vid_name = 'test'
impact_points = np.load(os.path.join(load_dir, f'{vid_name}_impact_points.npy'))
time_stamps = np.load(os.path.join(load_dir, f'{vid_name}_time_stamps.npy'))
print(time_stamps)
print(impact_points.shape)

[0 1 2 3 4 5 6 7 8 9]
(10, 4)


In [3]:
# function to quantize impact points
def quantize_impact_points(impact_points, time_stamps, quant_params):
    assert len(impact_points) == len(time_stamps)
    n_frames, n_limbs = impact_points.shape
    
    # quantize time stamps
    dt = 60 / (quant_params['bpm'] * quant_params['subdiv'])
    quantized_time_stamps = np.round((time_stamps - quant_params['start_time']) / dt) * dt + quant_params['start_time']
    quantized_time_stamps[quantized_time_stamps < 0] = 0
    
    # format quantized impacts
    quantized_impacts = []
    for l in range(n_limbs):
        quantized_impacts.append(quantized_time_stamps[impact_points[:,l]])
    
    return quantized_impacts

# testing
impact_points = np.random.choice([True, False], size=(10, 4))
print(impact_points)

quant_params = {
    'start_time': 0.25,
    'bpm': 120,
    'subdiv': 1
}
quantized_impacts = quantize_impact_points(impact_points, time_stamps, quant_params)
print(quantized_impacts)

[[ True False False False]
 [ True  True  True False]
 [False  True False  True]
 [False False  True False]
 [False False  True  True]
 [False  True  True  True]
 [False False  True False]
 [ True False  True False]
 [False False  True False]
 [False False False False]]
[array([0.25, 1.25, 7.25]), array([1.25, 2.25, 5.25]), array([1.25, 3.25, 4.25, 5.25, 6.25, 7.25, 8.25]), array([2.25, 4.25, 5.25])]


In [4]:
# save results to file
save_dir = os.path.join('data', 'quantized_impacts')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
vid_name = 'test'

with open(os.path.join(save_dir, f'{vid_name}_quantized_impacts.pkl'), 'wb') as f:
    pickle.dump(quantized_impacts, f)